In [1]:
import pandas as pd
from tqdm import tqdm
import re
from datetime import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [2]:
## 인코더
class Encoder(nn.Module):

    def __init__(self, input_size=4096, hidden_size=1024, num_layers=2):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True,
        #                     dropout=0.1, bidirectional=False)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=0.1, bidirectional=False)

    def forward(self, x):
        output, hidden = self.lstm(x)  # out: tensor of shape (batch_size, seq_length, hidden_size)

        return hidden
    
## 디코더
class Decoder(nn.Module):

    def __init__(self, input_size=4096, hidden_size=1024, output_size=4096, num_layers=2):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

#         self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True,
#                             dropout=0.1, bidirectional=False)
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=0.1, bidirectional=False)

        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden):
        output, hidden = self.lstm(x, hidden)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        prediction = self.fc(output)

        return prediction, hidden
    
## RNN Auto Encoder
class RNNAutoEncoder(nn.Module):

    def __init__(self,
                 input_dim: int,
                 latent_dim: int,
                 window_size: int=1,
                 **kwargs) -> None:
        """
        :param input_dim: 변수 Tag 갯수
        :param latent_dim: 최종 압축할 차원 크기
        :param window_size: 길이
        :param kwargs:
        """

        super(RNNAutoEncoder, self).__init__()

        self.latent_dim = latent_dim
        self.input_dim = input_dim
        self.window_size = window_size

        if "num_layers" in kwargs:
            num_layers = kwargs.pop("num_layers")
        else:
            num_layers = 1

        self.encoder = Encoder(
            input_size=input_dim,
            hidden_size=latent_dim,
            num_layers=num_layers,
        )
        self.reconstruct_decoder = Decoder(
            input_size=input_dim,
            output_size=input_dim,
            hidden_size=latent_dim,
            num_layers=num_layers,
        )

    def forward(self, src:torch.Tensor, **kwargs):
        batch_size, sequence_length, var_length = src.size()

        ## Encoder 넣기
        encoder_hidden = self.encoder(src)
        
        inv_idx = torch.arange(sequence_length - 1, -1, -1).long()
        reconstruct_output = []
        temp_input = torch.zeros((batch_size, 1, var_length), dtype=torch.float).to(src.device)
        hidden = encoder_hidden
        for t in range(sequence_length):
            temp_input, hidden = self.reconstruct_decoder(temp_input, hidden)
            reconstruct_output.append(temp_input)
        reconstruct_output = torch.cat(reconstruct_output, dim=1)[:, inv_idx, :]
        
        return [reconstruct_output, src]

    def loss_function(self,
                      *args,
                      **kwargs) -> dict:
        recons = args[0]
        input = args[1]
        
        ## MSE loss(Mean squared Error)
        loss =F.mse_loss(recons, input)
        return loss

In [3]:
from dataset import CurrentDataset

In [4]:
path = '/home/workspace/iot_ai_model/dataset/current/train/**/normal/*.csv'
dataset = CurrentDataset(path)

In [5]:
len(dataset)

68100

In [6]:
model = RNNAutoEncoder(3, 500, num_layers=2).cuda()

In [7]:
trn_dataloader = DataLoader(dataset,
                            batch_size=64,
                            shuffle=True,
                            num_workers=4,
                            pin_memory=True,
                            drop_last=False)

In [8]:
epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = F.mse_loss

In [ ]:
model.train()
hist_loss = []
pre_loss = 1.

In [10]:
for epoch in range(epochs):
    progress = tqdm(trn_dataloader)
    cost = 0.
    for path, cur, target in progress:
        cur = cur.cuda()
        output, _ = model(cur)
        loss = criterion(cur, output)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        progress.set_postfix_str(f'loss {loss.item()}')
        
        cost += loss.item()
    avg_loss = cost / len(train_loader)
    hist_loss.append(avg_loss)    
    progress.set_postfix_str(f'loss {avg_loss}')    

 32%|███▏      | 394/1217 [11:03<23:06,  1.68s/it, loss 98.58082580566406] 


KeyboardInterrupt: 

In [11]:
import numpy as np

In [13]:
np.random.random((1,300,300,3))

array([[[[0.08704378, 0.37334953, 0.7192736 ],
         [0.25851468, 0.08333939, 0.91786107],
         [0.61416842, 0.71758518, 0.56501066],
         ...,
         [0.18090796, 0.07895112, 0.93121052],
         [0.39633596, 0.10610315, 0.63402062],
         [0.31739749, 0.42837523, 0.71957548]],

        [[0.79688461, 0.81463725, 0.81951224],
         [0.43310532, 0.72843846, 0.3233063 ],
         [0.84818989, 0.36250963, 0.97919309],
         ...,
         [0.98370834, 0.01058058, 0.74192041],
         [0.57554875, 0.05285465, 0.73751619],
         [0.99109348, 0.69378585, 0.75760908]],

        [[0.57404126, 0.81415867, 0.01540538],
         [0.29621688, 0.64454995, 0.93091213],
         [0.80338385, 0.98140393, 0.24922374],
         ...,
         [0.00865711, 0.61437302, 0.94842847],
         [0.36159889, 0.27436632, 0.94589492],
         [0.75465225, 0.00206451, 0.67654211]],

        ...,

        [[0.24829576, 0.36673569, 0.04645826],
         [0.73450276, 0.58413347, 0.91377221]

In [11]:
import tensorflow as tf

In [1]:
import numpy as np

In [2]:
a = np.array([1,2,3,4,5,6,7,8,9,0])

In [5]:
a.reshape((None,2))

TypeError: 'NoneType' object cannot be interpreted as an integer

In [21]:
a = np.array([[1,2,4,12], [2,3,4,12]], dtype=np.float)
b = np.array([[[1,2,4,2], [1,2,4,5]], [[1,2,4,3], [1,2,4,3]]], dtype=np.float)

In [23]:
b.shape

(2, 2, 4)

In [26]:
tf.math.softmax(b, axis=-1)

<tf.Tensor: shape=(2, 2, 4), dtype=float64, numpy=
array([[[0.0377044 , 0.1024912 , 0.7573132 , 0.1024912 ],
        [0.01275478, 0.03467109, 0.25618664, 0.69638749]],

       [[0.0320586 , 0.08714432, 0.64391426, 0.23688282],
        [0.0320586 , 0.08714432, 0.64391426, 0.23688282]]])>

In [9]:
tf.squeeze(b)

<tf.Tensor: shape=(2, 2, 4), dtype=int64, numpy=
array([[[1, 2, 4, 2],
        [1, 2, 4, 5]],

       [[1, 2, 4, 3],
        [1, 2, 4, 3]]])>

In [22]:
a.shape

(2, 4)

In [36]:
np.take([1,2,4,5], [0,1], axis=0)

array([1, 2])

In [196]:
np.minimum(a,b)

array([[[1, 2, 4, 2],
        [1, 2, 4, 5]],

       [[1, 2, 4, 3],
        [1, 2, 4, 3]]])

In [199]:
import math

In [10]:
np.softmax

AttributeError: module 'numpy' has no attribute 'softmax'

In [ ]:
math

In [201]:
np.minimum([1,2,4,12], 0)

array([0, 0, 0, 0])

In [65]:
np.scatter_nd_update(a,b)

AttributeError: module 'numpy' has no attribute 'scatter_nd_update'

In [9]:
a = np.array([1,2,3,4,12])

In [12]:
tf.gather(a, [1,2,3,1,2,4,5])

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 2,  3,  4,  2,  3, 12,  0])>

In [13]:
np.take(a, [1,2,3])

array([2, 3, 4])

In [247]:
tf.math.reduce_any(a)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [249]:
a.astype(np.float)

array([ 1.,  2.,  3.,  4., 12.])

In [250]:
a

array([ 1,  2,  3,  4, 12])

In [251]:
np.expand_dims(a,0)

(1, 5)

In [248]:
np.any(a)

True

In [242]:
tf.argsort(a,direction='DESCENDING')

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 2, 1, 0], dtype=int32)>

In [243]:
(-a).argsort()

array([4, 3, 2, 1, 0])

In [231]:
np.argsort(a[:-1])

array([0, 1, 2, 3])

In [228]:
tf.argsort(a, direction='DESCENDING')

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 2, 1, 0], dtype=int32)>

In [207]:
tf.less(a, 3)

<tf.Tensor: shape=(5,), dtype=bool, numpy=array([ True,  True, False, False, False])>

In [220]:
np.zeros_like([3.,3.])

array([0., 0.])

In [218]:
tf.math.exp([3.,3.])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([20.085537, 20.085537], dtype=float32)>

In [208]:
a < 3

array([ True,  True, False, False, False])

In [47]:
np.expand_dims(a, axis=1).shape

(2, 1, 2, 4)

In [71]:
best_a = np.argmax(a, 1)
best_b = np.argmax(b, 1)

In [72]:
best_a, best_b

(array([[0, 0, 0, 0],
        [0, 0, 0, 0]]),
 array([[0, 0, 0, 1],
        [0, 0, 0, 0]]))

In [150]:
tensor = np.array([[1, 1], [2, 2], [3, 3]])
indices =  [[0, 1], [2, 0]]           # num_updates == 2, index_depth == 2
updates = [5, 10]     
print(tf.tensor_scatter_nd_update(tensor, indices, updates))

tf.Tensor(
[[ 1  5]
 [ 2  2]
 [10  3]], shape=(3, 2), dtype=int64)


In [195]:
np.ones_like(tensor, dtype=np.float32)

array([[1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)

In [192]:
np.ones_like(tensor

TypeError: Cannot interpret 'tf.float32' as a data type

In [188]:
def tensor_scatter_nd_update(tensor, indices, updates):
    for idx, coord in enumerate(indices):
        tensor[tuple(coord)] = updates[idx]

In [189]:
tensor_scatter_nd_update(tensor, indices, updates)

In [190]:
tensor

array([[ 1,  5],
       [ 2,  2],
       [10,  3]])

In [73]:
best_gt_idx = tf.tensor_scatter_nd_update(
        best_a,
        tf.expand_dims(best_b, 1),
        tf.range(best_b.shape[0], dtype=tf.int64))

InvalidArgumentError: Outer dimensions of indices and update must match. Indices shape: [2,1,4], updates shape:[2] [Op:TensorScatterUpdate]

In [ ]:
for 

In [50]:
np.arange(10, dtype=np.int64)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [49]:
tf.range(10, dtype=np.int64)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [38]:
a.shape

(2, 2, 4)

In [39]:
tf.expand_dims(a, 1)

<tf.Tensor: shape=(2, 1, 2, 4), dtype=int64, numpy=
array([[[[ 1,  2,  4, 12],
         [ 1,  2,  4, 12]]],


       [[[ 1,  2,  4, 12],
         [ 1,  2,  4, 12]]]])>

In [33]:
tf.math.reduce_max(a, 1)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([12, 12])>

In [30]:
np.max(a,1)

array([1234, 1234])

In [266]:
np.concatenate([[[1,2], [1,2]], [[1,2], [1,2]]], axis=-1)

array([[1, 2, 1, 2],
       [1, 2, 1, 2]])

In [267]:
tf.concat([[[1,2], [1,2]], [[1,2], [1,2]]], axis=-1)

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[1, 2, 1, 2],
       [1, 2, 1, 2]], dtype=int32)>

In [ ]:
np.concat